## Packages

In [ ]:
# ! pip install folium
# ! pip install shapely

import folium as fm
import chardet


import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 


from pandas import Series, DataFrame
from folium import Marker, GeoJson
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
from folium.plugins import MarkerCluster, HeatMap

import branca as br 

# Data Processing

In [ ]:
base = open(r'../../_data/institutos1.csv','rb').read()
det = chardet.detect(base)
charenc = det['encoding']

inst = pd.read_csv(r'../../_data/institutos1.csv', encoding = charenc)

inst_geo = gpd.GeoDataFrame(inst, crs = "EPSG:4326", 
                            geometry = gpd.points_from_xy(inst.nlong_ie, inst.nlat_ie))

shape = gpd.read_file(r'../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp')

intersct_inst_geo = gpd.overlay(inst_geo, shape, how = "intersection")

intersct_inst_geo

# Vacancies by department

In [ ]:
vars_sum = ['ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm',
            'sumaden_flagingresante',  'sumaden_flagmatriculado']

tot_dpt = intersct_inst_geo.groupby(['CCDD'], as_index = False )[vars_sum ].sum()

tot_dpt_shp = shape.merge(tot_dpt, on = 'CCDD' )

# government palace coordinates

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)

# Mandatory: geo_data in GeoJson format
# columns: variables from economics indicators data set
# Atention !!! key_on: commom variable between geodata and data "feature.properties.(name of variable)"

fm.Choropleth(
    geo_data=tot_dpt_shp,
    data=tot_dpt_shp,
    columns=['CCDD', 'ltimoden_metaatencion'],
    key_on="feature.properties.CCDD",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="Total_vacancies",
    smooth_factor=0,
    Highlight= True,
    line_color = "#0000",
    overlay=True,
    nan_fill_color = "White"  # fill white missing values 
    ).add_to(z)

#fm.LayerControl().add_to(z)

# Save in a html format 
z.save(r"./group_5_ass_9_html_maps/Total_vacancies.html")
z

# Radio by vacancies - Surplus

In [ ]:
radio_vac_map_s = fm.Map(location = ubication, zoom_start = 5, control_scale=True)

# get superior ratios
rads_s = list(up_ratio['ratio_applicants_vacancies'])

# Create feature group
feature_group = fm.FeatureGroup('Applicant Surplus')

for i in range(len(up_coord)):
    fm.Circle(up_coord[i], radius = rads_s[i]*10000,color = 'forestgreen').add_to(feature_group)

# Agregate feature to map    
feature_group.add_to(radio_vac_map_s)

# Layer contro
fm.LayerControl(collapsed=False).add_to(radio_vac_map_s)

radio_vac_map_s.save(r"./group_5_ass_9_html_maps/radio_vac_map_s.html")
radio_vac_map_s  

# Radio by vacancies - Deficit

In [ ]:
radio_vac_map_d = fm.Map(location = ubication, zoom_start = 5, control_scale=True)

# Generate inferior radious
rads_d = list(down_ratio['ratio_applicants_vacancies'])

# Feature group
feature_group = fm.FeatureGroup('Applicant Deficit')

for i in range(len(down_coord)):
    fm.Circle(down_coord[i], radius = rads_d[i]*10000,color = 'red').add_to(feature_group)
    
feature_group.add_to(radio_vac_map_d)
    
fm.LayerControl(collapsed=False).add_to(radio_vac_map_d)

radio_vac_map_d.save(r"./group_5_ass_9_html_maps/radio_vac_map_d.html")
radio_vac_map_d